# Team Members: Kevon C., Payal M., Kevin S.

## Movie Genre Prediction

### Part 2: Descriptive Stats

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import time
from collections import defaultdict, Counter

C:\Users\munip\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
#import statements

import pandas as pd
import re

import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
#nltk.download('averaged_perceptron_tagger')

from collections import Counter, defaultdict

import nltk
nltk.download('stopwords')
from typing import Iterable


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\munip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Reading in Movie CSV Files as Data Frames

In [3]:
#Reading in movies_dataset.csv

movies_dataset_df = pd.read_csv('movies_dataset.csv')

In [4]:
movies_dataset_df.head()

,Unnamed: 0,title,year,advisory,movie_length,genre,rating,metascore,description,votes
0,0,7.Thor: Love and Thunder(2022),(2022),PG-13,118 min,"Action, Adventure, Comedy",6.4,57 \n Metascore,"Thor enlists the help of Valkyrie, Korg and ex...","Votes:\n275,367"
1,1,12.Avatar(2009),(2009),PG-13,162 min,"Action, Adventure, Fantasy",7.8,83 \n Metascore,A paraplegic Marine dispatched to the moon Pan...,"Votes:\n1,231,498\n| Gross:\n$760.51M"
2,2,14.DC League of Super-Pets(2022),(2022),PG,105 min,"Animation, Action, Adventure",7.4,56 \n Metascore,Krypto the Super-Dog and Superman are insepara...,"Votes:\n52,463"
3,3,15.Pinocchio(I) (2022),(I) (2022),PG,105 min,"Adventure, Comedy, Drama",5.1,38 \n Metascore,"A puppet is brought to life by a fairy, who as...","Votes:\n27,171"
4,4,17.The Greatest Beer Run Ever(2022),(2022),R,126 min,"Adventure, Comedy, Drama",6.8,39 \n Metascore,A man's story of leaving New York in 1967 to b...,"Votes:\n5,460"


In [5]:
#Reading in top_movies_dataset.csv

top_movies_dataset_df = pd.read_csv('top_movies_dataset.csv')

In [6]:
top_movies_dataset_df.head()

,Unnamed: 0,title,year,advisory,movie_length,genre,rating,description,metascore
0,0,The Shawshank Redemption,1994,R,2h 22m,Drama,9.3,Two imprisoned men bond over a number of years...,81
1,1,The Godfather,1972,R,2h 55m,CrimeDrama,9.2,The aging patriarch of an organized crime dyna...,100
2,2,The Dark Knight,2008,PG-13,2h 32m,ActionCrimeDrama,9.0,When the menace known as the Joker wreaks havo...,84
3,3,The Godfather Part II,1974,R,3h 22m,CrimeDrama,9.0,The early life and career of Vito Corleone in ...,90
4,4,12 Angry Men,1957,Approved,1h 36m,CrimeDrama,9.0,The jury in a New York City murder trial is fr...,96


In [7]:
#seeing that the top_movies_dataset_df does not have the column, votes,
#and that the column Votes in the movies_dataset_df does not provide 
#any information that would contribute to classifying genre,
#we will drop the votes column from the movies_dataset_df dataframe

movies_dataset_df = movies_dataset_df.drop('votes', axis = 1)

In [8]:
#will concat the two movie dataframes into one file
#given that column names are all the same

movies_df = pd.concat([movies_dataset_df,top_movies_dataset_df], ignore_index=True)

In [9]:
movies_df.head()

,Unnamed: 0,title,year,advisory,movie_length,genre,rating,metascore,description
0,0,7.Thor: Love and Thunder(2022),(2022),PG-13,118 min,"Action, Adventure, Comedy",6.4,57 \n Metascore,"Thor enlists the help of Valkyrie, Korg and ex..."
1,1,12.Avatar(2009),(2009),PG-13,162 min,"Action, Adventure, Fantasy",7.8,83 \n Metascore,A paraplegic Marine dispatched to the moon Pan...
2,2,14.DC League of Super-Pets(2022),(2022),PG,105 min,"Animation, Action, Adventure",7.4,56 \n Metascore,Krypto the Super-Dog and Superman are insepara...
3,3,15.Pinocchio(I) (2022),(I) (2022),PG,105 min,"Adventure, Comedy, Drama",5.1,38 \n Metascore,"A puppet is brought to life by a fairy, who as..."
4,4,17.The Greatest Beer Run Ever(2022),(2022),R,126 min,"Adventure, Comedy, Drama",6.8,39 \n Metascore,A man's story of leaving New York in 1967 to b...


In [10]:
#removing unamed column
#it is the same as the index

movies_df = movies_df.drop('Unnamed: 0', axis = 1)

In [11]:
#seeing how many rows in current dataframe
print("There are", len(movies_df.axes[0]), "movies in this dataframe." )

There are 692 movies in this dataframe.


In [12]:
#removing duplicates in case there are any
movies_df = movies_df.drop_duplicates(keep = 'first', ignore_index = True)

In [13]:
#seeing if movie count changed after dropping potential 
#duplicates
print("After removing duplicates, there are", len(movies_df.axes[0]), 
      "movies in this dataframe." )

After removing duplicates, there are 678 movies in this dataframe.


In [14]:
#for title, removing the number .
#format (example: remove 7.)
#source: https://stackoverflow.com/questions/12851791/
#removing-numbers-from-string

title = []

for row in movies_df['title']:
  result = ''.join([i for i in row if not i.isdigit()])
  title.append(result)


In [15]:
movies_df['title'] = title

In [16]:
movies_df.head()

,title,year,advisory,movie_length,genre,rating,metascore,description
0,.Thor: Love and Thunder(),(2022),PG-13,118 min,"Action, Adventure, Comedy",6.4,57 \n Metascore,"Thor enlists the help of Valkyrie, Korg and ex..."
1,.Avatar(),(2009),PG-13,162 min,"Action, Adventure, Fantasy",7.8,83 \n Metascore,A paraplegic Marine dispatched to the moon Pan...
2,.DC League of Super-Pets(),(2022),PG,105 min,"Animation, Action, Adventure",7.4,56 \n Metascore,Krypto the Super-Dog and Superman are insepara...
3,.Pinocchio(I) (),(I) (2022),PG,105 min,"Adventure, Comedy, Drama",5.1,38 \n Metascore,"A puppet is brought to life by a fairy, who as..."
4,.The Greatest Beer Run Ever(),(2022),R,126 min,"Adventure, Comedy, Drama",6.8,39 \n Metascore,A man's story of leaving New York in 1967 to b...


In [17]:
#removing () from end of title string
#source for checking last 2
#characters of string: https://thispointer.com/python-how-to-get-last-n-
#characters-in-a-string/
#source for removing last 2 characters of string:
#https://stackoverflow.com/questions/15478127/remove-
#final-character-from-string

title = []

for row in movies_df['title']:
  if row[-2:] == "()":
    result = row[:-2]
    title.append(result)
  else:
    title.append(row)

In [18]:
movies_df['title'] = title

In [19]:
title = []

for row in movies_df['title']:
  if row[0:] == ".":
    result = row[1:]
    title.append(result)
  else:
    title.append(row)

In [20]:
movies_df['title'] = title

In [21]:
#randomly checking rows to ensure title has been cleaned
movies_df.sample(n=5)

,title,year,advisory,movie_length,genre,rating,metascore,description
549,,2019,R,1h 59m,ActionDramaWar,8.2,78,"April 6th, 1917. As an infantry battalion asse..."
644,Spotlight,2015,R,2h 9m,BiographyCrimeDrama,8.1,93,The true story of how the Boston Globe uncover...
390,.Legends of the Fall,(1994),R,133 min,"Drama, Romance, War",7.5,45 \n Metascore,"In the early 1900s, three brothers and their f..."
441,The Lord of the Rings: The Two Towers,2002,PG-13,2h 59m,ActionAdventureDrama,8.8,87,While Frodo and Sam edge closer to Mordor with...
319,.Warrior,(2011),PG-13,140 min,"Action, Drama, Sport",8.2,71 \n Metascore,The youngest son of an alcoholic former boxer ...


In [22]:


#flatten function from assignment 3 (with og
#source also listed)

#flatten source: #https://stackoverflow.com/questions/952914/
#how-do-i-make-a-flat-list-out-of-a-list-of-lists

def flatten(items):
    """Yield items from any nested iterable; see Reference."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
            yield x


In [23]:
#removing () from year
#source: https://stackoverflow.com/questions/56375850/how-to-remove-parentheses
#-from-string
year = []

for row in movies_df['year']:
  row = str(row)
  result = re.sub(r'[()]', '', row)
  year.append(result)


In [24]:
movies_df['year'] = year

In [25]:
#randomly checking rows to ensure year has no ()
movies_df.sample(n=5)

,title,year,advisory,movie_length,genre,rating,metascore,description
488,The Shining,1980,R,2h 26m,DramaHorror,8.4,66,A family heads to an isolated hotel for the wi...
573,The Treasure of the Sierra Madre,1948,Passed,2h 6m,AdventureDramaWestern,8.2,98,Two down-on-their-luck Americans searching for...
385,.Tropic Thunder,2008,R,107 min,"Action, Comedy, War",7.0,71 \n Metascore,"Through a series of freak occurrences, a group..."
631,"Monsters, Inc.",2001,G,1h 32m,AnimationAdventureComedy,8.1,79,"In order to power the city, monsters have to s..."
412,.Bone Tomahawk,2015,Not Rated,132 min,"Drama, Horror, Western",7.1,72 \n Metascore,"In the dying days of the old west, an elderly ..."


In [26]:
#removing I & II from front of year
year = []

year = []

for row in movies_df['year']:
  if row[0:] == "II":
    result = row[2:]
    year.append(result)
  else:
    year.append(row)

In [27]:
movies_df['year'] = year

In [28]:
movies_df.head()

,title,year,advisory,movie_length,genre,rating,metascore,description
0,.Thor: Love and Thunder,2022,PG-13,118 min,"Action, Adventure, Comedy",6.4,57 \n Metascore,"Thor enlists the help of Valkyrie, Korg and ex..."
1,.Avatar,2009,PG-13,162 min,"Action, Adventure, Fantasy",7.8,83 \n Metascore,A paraplegic Marine dispatched to the moon Pan...
2,.DC League of Super-Pets,2022,PG,105 min,"Animation, Action, Adventure",7.4,56 \n Metascore,Krypto the Super-Dog and Superman are insepara...
3,.Pinocchio(I),I 2022,PG,105 min,"Adventure, Comedy, Drama",5.1,38 \n Metascore,"A puppet is brought to life by a fairy, who as..."
4,.The Greatest Beer Run Ever,2022,R,126 min,"Adventure, Comedy, Drama",6.8,39 \n Metascore,A man's story of leaving New York in 1967 to b...


In [29]:
#seeing how movie length is recorded
#will turn these all into minutes
movies_df['movie_length'].sample(10)

406    105 min
192    105 min
667     1h 39m
541     2h 11m
293    112 min
127    113 min
18     147 min
467     1h 46m
53     130 min
454      3h 9m
Name: movie_length, dtype: object

In [30]:
#Movie: The Boat led to error messages due to incorrect
#recording of data
#so I will remove it from the movies_df

#source for dropping index:
#https://stackoverflow.com/questions/43136137/drop-a-specific-row-in-pandas

#removing row: The Boat
the_boat_index = movies_df[(movies_df['title'] == 'The Boat')].index
#dropping the movie:
movies_df = movies_df.drop(the_boat_index)

In [31]:
#changing movie length to minutes
#source: https://datascience.stackexchange.com/questions/46909/
#how-to-convert-a-pandas-column-having-duration-details-in-string-format-ex1hr
movie_length = []

for row in movies_df['movie_length']:
  if row[-3:] == "min":
    result = row[:-3]
    result = int(result)
    movie_length.append(result)

  elif(row[-1:] == "h"):
    result = row[:-1]
    result = int(result) * 60
    movie_length.append(result)

  else:
    tester = (row.replace("h",'*60'))
    tester = tester.replace('m','*1')
    tester = tester.replace(' ', '+')
    tester = eval(tester)
    movie_length.append(tester)


In [33]:
movies_df['movie_length'] = movie_length

In [34]:
movies_df.sample(n=5)

,title,year,advisory,movie_length,genre,rating,metascore,description
595,Warrior,2011,PG-13,140,ActionDramaSport,8.2,71,The youngest son of an alcoholic former boxer ...
88,.The Godfather,1972,R,175,"Crime, Drama",9.2,100 \n Metascore,The aging patriarch of an organized crime dyna...
377,.The Last of the Mohicans,1992,R,112,"Action, Adventure, Drama",7.7,76 \n Metascore,Three trappers protect the daughters of a Brit...
496,The Dark Knight Rises,2012,PG-13,164,ActionDrama,8.4,78,Eight years after the Joker's reign of anarchy...
65,.The Munsters,2022,PG,109,"Comedy, Family, Fantasy",4.6,57 \n Metascore,"Reboot of ""The Munsters"", that followed a fami..."


In [35]:
#for genre, some of the words are not 
#comma separated
#the goal is to separate genres
#we will then tokenize this afterwards
#in a separate column

#source: https://stackoverflow.com/questions/16380326/check-if-
#substring-is-in-a-list-of-strings
genres = ["Adventure", "Action", "Animation", "Biography", "Comedy", 
          "Crime", "Drama", "Family", "Fantasy", "Film-Noir", "History", 
          "Horror", "Music", "Musical", "Mystery", "Romance", "Sci-Fi", 
          "Sport", "Thriller", "War", "Western"]



In [36]:
genre_sep = []

for row in movies_df['genre']:
  temp_list = []
  for genre in genres:
    if(genre in row):
      temp_list.append(genre)
  genre_sep.append(temp_list)


In [37]:
movies_df['sep_genre'] = genre_sep

In [38]:
movies_df.sample(n=5)

,title,year,advisory,movie_length,genre,rating,metascore,description,sep_genre
508,Once Upon a Time in America,1984,R,229,CrimeDrama,8.3,75,A former Prohibition-era Jewish gangster retur...,"[Crime, Drama]"
36,.Dahmer,2002,R,101,"Biography, Crime, Drama",5.6,63 \n Metascore,Biopic about notorious American serial killer ...,"[Biography, Crime, Drama]"
48,.The Greatest Showman,2017,PG,105,"Biography, Drama, Musical",7.5,48 \n Metascore,Celebrates the birth of show business and tell...,"[Biography, Drama, Music, Musical]"
339,.The Art of Racing in the Rain,2019,PG,109,"Comedy, Drama, Romance",7.5,43 \n Metascore,"Through his bond with his owner, aspiring Form...","[Comedy, Drama, Romance]"
192,.The Invitation(II),II 2022,PG-13,105,"Horror, Thriller",5.2,45 \n Metascore,A young woman is courted and swept off her fee...,"[Horror, Thriller]"


In [39]:
#removing the \nMetascore from the metascore column
#source for new_words code:
#https://stackoverflow.com/questions/29771168/
#how-to-remove-words-from-a-list-in-python

clean_metascore = []

for row in movies_df['metascore']:
  row = str(row).split()
  new_words = list(filter(lambda w: w not in 'Metascore', row))
  clean_metascore.append(new_words)

In [40]:
movies_df['new_metascore'] = clean_metascore

In [41]:
movies_df.head(n=10)

,title,year,advisory,movie_length,genre,rating,metascore,description,sep_genre,new_metascore
0,.Thor: Love and Thunder,2022,PG-13,118,"Action, Adventure, Comedy",6.4,57 \n Metascore,"Thor enlists the help of Valkyrie, Korg and ex...","[Adventure, Action, Comedy]",[57]
1,.Avatar,2009,PG-13,162,"Action, Adventure, Fantasy",7.8,83 \n Metascore,A paraplegic Marine dispatched to the moon Pan...,"[Adventure, Action, Fantasy]",[83]
2,.DC League of Super-Pets,2022,PG,105,"Animation, Action, Adventure",7.4,56 \n Metascore,Krypto the Super-Dog and Superman are insepara...,"[Adventure, Action, Animation]",[56]
3,.Pinocchio(I),I 2022,PG,105,"Adventure, Comedy, Drama",5.1,38 \n Metascore,"A puppet is brought to life by a fairy, who as...","[Adventure, Comedy, Drama]",[38]
4,.The Greatest Beer Run Ever,2022,R,126,"Adventure, Comedy, Drama",6.8,39 \n Metascore,A man's story of leaving New York in 1967 to b...,"[Adventure, Comedy, Drama]",[39]
5,.Everything Everywhere All at Once,2022,R,139,"Action, Adventure, Comedy",8.1,81 \n Metascore,An aging Chinese immigrant is swept up in an i...,"[Adventure, Action, Comedy]",[81]
6,.The Lord of the Rings: The Fellowship of the ...,2001,PG-13,178,"Action, Adventure, Drama",8.8,92 \n Metascore,A meek Hobbit from the Shire and eight compani...,"[Adventure, Action, Drama]",[92]
7,.Jurassic World Dominion,2022,PG-13,147,"Action, Adventure, Sci-Fi",5.7,38 \n Metascore,Four years after the destruction of Isla Nubla...,"[Adventure, Action, Sci-Fi]",[38]
8,.Rogue One: A Star Wars Story,2016,PG-13,133,"Action, Adventure, Sci-Fi",7.8,65 \n Metascore,"In a time of conflict, a group of unlikely her...","[Adventure, Action, Sci-Fi]",[65]
9,.Prey(I),I 2022,R,99,"Action, Adventure, Drama",7.2,71 \n Metascore,"Naru, a skilled warrior of the Comanche Nation...","[Adventure, Action, Drama]",[71]


In [42]:
#removing brackets in new metascore
movies_df['new_metascore'] = movies_df['new_metascore'].str[0]

In [43]:
movies_df.head(10)

,title,year,advisory,movie_length,genre,rating,metascore,description,sep_genre,new_metascore
0,.Thor: Love and Thunder,2022,PG-13,118,"Action, Adventure, Comedy",6.4,57 \n Metascore,"Thor enlists the help of Valkyrie, Korg and ex...","[Adventure, Action, Comedy]",57
1,.Avatar,2009,PG-13,162,"Action, Adventure, Fantasy",7.8,83 \n Metascore,A paraplegic Marine dispatched to the moon Pan...,"[Adventure, Action, Fantasy]",83
2,.DC League of Super-Pets,2022,PG,105,"Animation, Action, Adventure",7.4,56 \n Metascore,Krypto the Super-Dog and Superman are insepara...,"[Adventure, Action, Animation]",56
3,.Pinocchio(I),I 2022,PG,105,"Adventure, Comedy, Drama",5.1,38 \n Metascore,"A puppet is brought to life by a fairy, who as...","[Adventure, Comedy, Drama]",38
4,.The Greatest Beer Run Ever,2022,R,126,"Adventure, Comedy, Drama",6.8,39 \n Metascore,A man's story of leaving New York in 1967 to b...,"[Adventure, Comedy, Drama]",39
5,.Everything Everywhere All at Once,2022,R,139,"Action, Adventure, Comedy",8.1,81 \n Metascore,An aging Chinese immigrant is swept up in an i...,"[Adventure, Action, Comedy]",81
6,.The Lord of the Rings: The Fellowship of the ...,2001,PG-13,178,"Action, Adventure, Drama",8.8,92 \n Metascore,A meek Hobbit from the Shire and eight compani...,"[Adventure, Action, Drama]",92
7,.Jurassic World Dominion,2022,PG-13,147,"Action, Adventure, Sci-Fi",5.7,38 \n Metascore,Four years after the destruction of Isla Nubla...,"[Adventure, Action, Sci-Fi]",38
8,.Rogue One: A Star Wars Story,2016,PG-13,133,"Action, Adventure, Sci-Fi",7.8,65 \n Metascore,"In a time of conflict, a group of unlikely her...","[Adventure, Action, Sci-Fi]",65
9,.Prey(I),I 2022,R,99,"Action, Adventure, Drama",7.2,71 \n Metascore,"Naru, a skilled warrior of the Comanche Nation...","[Adventure, Action, Drama]",71


In [59]:
#source: from chapter 11 of textbook

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    #text = " ".join(text)
    return(text)

In [60]:
movies_df['clean_description'] = movies_df['description'].apply(clean_text)

In [61]:
movies_df.sample(n=5)

,title,year,advisory,movie_length,genre,rating,metascore,description,sep_genre,new_metascore,clean_description
658,Jai Bhim,2021,TV-MA,164,CrimeDramaMystery,8.9,96,When a tribal man is arrested for a case of al...,"[Crime, Drama, Mystery]",96,"[tribal, man, arrested, case, alleged, theft, ..."
344,.Don't Worry Darling(I),I 2022,R,123,"Drama, Mystery, Thriller",6.3,48 \n Metascore,A 1950s housewife living with her husband in a...,"[Drama, Mystery, Thriller]",48,"[housewife, living, husband, utopian, experime..."
358,.Glass Onion: A Knives Out Mystery,2022,PG-13,139,"Crime, Drama, Mystery",8.0,81 \n Metascore,Famed Southern detective Benoit Blanc travels ...,"[Crime, Drama, Mystery]",81,"[famed, southern, detective, benoit, blanc, tr..."
299,.Nope,2022,R,130,"Horror, Mystery, Sci-Fi",6.9,77 \n Metascore,The residents of a lonely gulch in inland Cali...,"[Horror, Mystery, Sci-Fi]",77,"[residents, lonely, gulch, inland, california,..."
424,.Maverick,1994,PG,127,"Action, Adventure, Comedy",7.0,62 \n Metascore,"Bret Maverick, needing money for a poker tourn...","[Adventure, Action, Comedy]",62,"[bret, maverick, needing, money, poker, tourna..."


In [62]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
  """
  Given a list of tokens, print number of tokens, number of unique
  number of characters, lexical diversity (https://en.wikipedia.or
  and num_tokens most common tokens. Return a list with the number
  of unique tokens, lexical diversity, and number of characters.
  """
  if verbose :
    print(f"There are {len(tokens)} tokens in the data.")
    print(f"There are {len(set(tokens))} unique tokens in the data.")
    print(f"There are {len(''.join(tokens))} characters in the data.")
    print(f"The lexical diversity is {len(set(tokens))/len(tokens)}.")

    counts = Counter(tokens)
    if num_tokens > 0 :
      print(counts.most_common(num_tokens))

  return([len(tokens),
    len(set(tokens)),
    len("".join(tokens)),
    len(set(tokens))/len(tokens)])

In [63]:
# We need to transform the lyrics to lists of tokens for descriptive statistics
token_df = movies_df['clean_description']
token_df_2 = list(flatten(token_df))

In [65]:
descriptive_stats(token_df_2)

There are 17707 tokens in the data.
There are 3992 unique tokens in the data.
There are 114589 characters in the data.
The lexical diversity is 0.2254475631106342.
[('young', 134), ('life', 116), ('war', 101), ('new', 93), ('help', 88)]


[17707, 3992, 114589, 0.2254475631106342]